# Regime-Adaptive Agent: End-to-End Demo

This notebook demonstrates the core reasoning loop of the trading research agent.

1.  **Setup**: Load libraries and configuration.
2.  **Perceive**: Ingest market (yfinance) and macro (FRED) data.
3.  **Analyze**: Compute features and detect the current market regime.
4.  **Baseline**: Run a backtest with a default 'vanilla' strategy.
5.  **Plan**: Use the Gemini LLM to propose alternative strategy parameters based on the regime.
6.  **Test & Evaluate**: Execute the backtests proposed by the agent and compare results.

## 1. Setup & Imports

In [ ]:
import pandas as pd
import warnings
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Suppress unnecessary warnings
warnings.filterwarnings('ignore', category=FutureWarning)
pd.set_option('display.float_format', '{:.4f}'.format)

# Import our custom modules
from src.utils.config import config
from src.data.ingest import fetch_ohlcv_data, fetch_fred_data
from src.features.engine import compute_features
from src.features.regime import detect_regime
from src.backtest.runner import run_backtest
from src.agent.planner import propose_actions

print("Setup complete. Configuration loaded:")
print(f"Reference Asset: {config['reference_asset']}")
print(f"Universe: {config['universe']}")

## 2. Perceive: Ingest Data

In [ ]:
# Fetch all required OHLCV and FRED data
# The function will use cached data if available, or download if not.
ohlcv_data = fetch_ohlcv_data()
fred_data = fetch_fred_data()

ref_asset = config['reference_asset']
print(f"\n{ref_asset} data (last 5 days):")
display(ohlcv_data[ref_asset].tail())

## 3. Analyze: Compute Features & Detect Regime

In [ ]:
# Compute features for our reference asset
features_df = compute_features(ohlcv_data, ref_asset, config['vix_ticker'])

# Detect the current regime based on the latest features
current_regime = detect_regime(features_df)

print("Latest Features:")
display(features_df.iloc[-1:].T)
print(f"\n---> Current Detected Regime: {current_regime}")

## 4. Baseline: Run Backtest with Default Strategy

We run a standard momentum strategy to establish a performance baseline. The agent's goal will be to improve upon this.

In [ ]:
# Get the default strategy configuration
baseline_strategy = config['strategies'][0]
strategy_name = baseline_strategy['name']
baseline_params = baseline_strategy['default_params']

print(f"Running baseline backtest for '{strategy_name}' on {ref_asset}...")
print(f"Parameters: {baseline_params}")

baseline_stats = run_backtest(
    ohlcv_df=ohlcv_data[ref_asset],
    asset_ticker=ref_asset,
    strategy_name=strategy_name,
    params=baseline_params
)

print("\n--- Baseline Performance --- ")
display(baseline_stats)

## 5. Plan: Ask the Gemini Agent for Suggestions

In [ ]:
# Check if Google API key is available before proceeding
if not os.getenv("GOOGLE_API_KEY"):
    print("GOOGLE_API_KEY not found in .env file. Skipping planner step.")
    llm_proposals = []
else:
    # Now, we pass the context (regime, features, baseline) to the LLM planner
    llm_proposals = propose_actions(
        regime=current_regime,
        features_df=features_df,
        baseline_stats=baseline_stats
    )

## 6. Test & Evaluate Agent's Proposals

Finally, we execute the backtests suggested by the LLM and compare them against our baseline.

In [ ]:
all_results = []

# Add baseline result for comparison
baseline_result = baseline_stats.to_dict()
baseline_result['label'] = 'Baseline'
baseline_result['params'] = str(baseline_params)
all_results.append(baseline_result)

if not llm_proposals:
    print("No proposals from LLM to test.")
else:
    print("\n--- Testing LLM Proposals ---")
    for i, proposal in enumerate(llm_proposals):
        print(f"\nRunning test for Proposal #{i+1}: {proposal['params']}")
        
        # Run the backtest for the proposed parameters
        proposal_stats = run_backtest(
            ohlcv_df=ohlcv_data[proposal['asset_ticker']],
            asset_ticker=proposal['asset_ticker'],
            strategy_name=proposal['strategy_name'],
            params=proposal['params']
        )
        
        if proposal_stats is not None:
            display(proposal_stats)
            proposal_result = proposal_stats.to_dict()
            proposal_result['label'] = f'LLM_Proposal_{i+1}'
            proposal_result['params'] = str(proposal['params'])
            all_results.append(proposal_result)
        else:
            print("Backtest failed for this proposal.")

# Create a final comparison dataframe
results_df = pd.DataFrame(all_results).set_index('label')
results_df = results_df[['Total Return [%]', 'Sharpe Ratio', 'Max Drawdown [%]', 'Num Trades', 'params']]

print("\n--- Final Comparison --- ")
display(results_df.sort_values('Sharpe Ratio', ascending=False))